<a href="https://colab.research.google.com/github/BoJavs-svg/LLM_Lora_FineTunning/blob/main/nb/Qwen3_(14B)-Reasoning-Conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth
    !pip install fastapi uvicorn pyngrok nest-asyncio

from huggingface_hub import login
from google.colab import userdata
login(userdata.get('hf'))

### Unsloth

In [2]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

# 1. Load the base model
base_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-14B",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

# 2. Reapply LoRA the same way as originally done
model = FastLanguageModel.get_peft_model(
    base_model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

# 3. Load adapter weights directly
# model.load_adapter("BoJavs/TrainedQwen2.5", adapter_name="default")
# model.set_adapter("default")
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<|im_start|>", "<|im_end|>"]
})

# Set the chat template manually (Jinja2-style template)
tokenizer.chat_template = """{% for message in messages %}
{{ '<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>\n' }}
{% endfor %}
{% if add_generation_prompt %}
<|im_start|>assistant
{% endif %}"""
# 4. Continue with training or inference
print(sum(p.requires_grad for p in model.parameters()))  # Should be > 0


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.9: Fast Qwen2 patching. Transformers: 4.52.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/196k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

Unsloth 2025.5.9 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


672


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


<a name="Data"></a>
### Data Prep
Qwen3 has both reasoning and a non reasoning mode. So, we should use 2 datasets:

1. We use the [Open Math Reasoning]() dataset which was used to win the [AIMO](https://www.kaggle.com/competitions/ai-mathematical-olympiad-progress-prize-2/leaderboard) (AI Mathematical Olympiad - Progress Prize 2) challenge! We sample 10% of verifiable reasoning traces that used DeepSeek R1, and whicht got > 95% accuracy.

2. We also leverage [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we need to convert it to HuggingFace's normal multiturn format as well.

In [4]:
from datasets import load_dataset
swe_bench_lite = load_dataset('BoJavs/Clean_SweBench', split='train')

Let's see the structure of both datasets:

In [5]:
swe_bench_lite

Dataset({
    features: ['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit', 'image_name'],
    num_rows: 176
})

Next we take the non reasoning dataset and convert it to conversational format as well.

> Agregar bloque entrecomillado



We have to use Unsloth's `standardize_sharegpt` function to fix up the format of the dataset first.

In [6]:
def separate_patch(patch_text):
    old_lines = []
    new_lines = []

    for line in patch_text.splitlines():
        if line.startswith('@@'):
            # diff hunk header, ignore or you can parse line ranges if needed
            continue
        elif line.startswith('-'):
            # removed line: add to old code only
            old_lines.append(line[1:])
        elif line.startswith('+'):
            # added line: add to new code only
            new_lines.append(line[1:])
        else:
            # context line, add to both old and new
            old_lines.append(line)
            new_lines.append(line)

    old_code = "\n".join(old_lines)
    new_code = "\n".join(new_lines)

    return old_code, new_code


In [7]:
def generate_conversation(batch):
  conversations = []

  for problem, patch, repo in zip(
      batch["problem_statement"],
      batch["patch"],
      batch["repo"],
  ):
    prev, new = separate_patch(patch)
    user_prompt = f"""\
  We're currently solving the following issue within our repository. Here's the issue text:
  ISSUE:
  {problem}
  Now, you're going to solve this issue on your own.
  The issue is in:
  {prev}
  YOU MUST RETURN A PATCH
          """
    patch_prompt= f"""\
  DISCUSSION
  The solved code for this problem is:
  {new}
  <command>
  {patch}
  </command>
          """
    conversations.append([
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": patch_prompt}
        ])

  return {"conversations": conversations}


In [8]:
from unsloth.chat_templates import standardize_sharegpt

# Standardize the dataset first
dataset = standardize_sharegpt(swe_bench_lite)

# Apply chat template with explicit thinking mode
swe_bench_lite_conversations = tokenizer.apply_chat_template(
    dataset.map(generate_conversation, batched = True)["conversations"],
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # Explicitly disable thinking mode
)

Let's see the first row

In [9]:
swe_bench_lite_conversations[1]


'<|im_start|>user\n  We\'re currently solving the following issue within our repository. Here\'s the issue text:\n  ISSUE:\n  "default.html" deprecation warning raised for ManagementForm\'s\nDescription\n\t\nI have a project where I never render forms with the {{ form }} expression. However, I\'m still getting the new template deprecation warning because of the formset management form production, during which the template used is insignificant (only hidden inputs are produced).\nIs it worth special-casing this and avoid producing the warning for the management forms?\n\n  Now, you\'re going to solve this issue on your own.\n  The issue is in:\n  diff --git a/django/forms/formsets.py b/django/forms/formsets.py\n-- a/django/forms/formsets.py\n     as well.\n     """\n \n     TOTAL_FORMS = IntegerField(widget=HiddenInput)\n     INITIAL_FORMS = IntegerField(widget=HiddenInput)\n     # MIN_NUM_FORM_COUNT and MAX_NUM_FORM_COUNT are output with the rest of the\n  YOU MUST RETURN A PATCH\n    

Now let's see how long both datasets are:

In [10]:
print(len(swe_bench_lite_conversations))

176


Finally combine both datasets:

In [11]:
import pandas as pd
data = pd.Series(swe_bench_lite_conversations)
data.name = "text"

from datasets import Dataset
dataset = Dataset.from_pandas(pd.DataFrame(data))
dataset = dataset.shuffle(seed = 3407)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [12]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        # warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 15,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/176 [00:00<?, ? examples/s]

Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [13]:
for _ in range(3):
  trainer_stats = trainer.train()
  model.save_pretrained("lora_model")
  tokenizer.save_pretrained("lora_model")
  model.push_to_hub_merged(
      "BoJavs/TrainedQwen2.5",
      tokenizer,
      save_method="lora",
  )

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 176 | Num Epochs = 1 | Total steps = 22
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 137,625,600/14,000,000,000 (0.98% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.255900
2,1.021700
3,1.077200
4,1.173400
5,1.246200
6,0.863600
7,0.895000
8,1.177400
9,0.746300
10,0.940400


Unsloth: Saving LoRA adapters. Please wait...


adapter_model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 176 | Num Epochs = 1 | Total steps = 22
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 137,625,600/14,000,000,000 (0.98% trained)


Step,Training Loss
1,0.658800
2,0.542800
3,0.521900
4,0.626700
5,0.717500
6,0.525900
7,0.556600
8,0.796200
9,0.550600
10,0.656400


Unsloth: Saving LoRA adapters. Please wait...


adapter_model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

Saved lora model to https://huggingface.co/BoJavs/TrainedQwen2.5


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [14]:
# # model.save_pretrained_merged(
# #     "merged_model",
# #     tokenizer,
# #     save_method="merged_16bit",
# # )

# # model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")
# model.push_to_hub_gguf(
#     "BoJavs/TrainedQwen2.5-GGUF",
#     tokenizer,
#     quantization_method=["q4_k_m"],
# )


Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 12.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 51.97 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 81%|████████▏ | 39/48 [00:00<00:00, 55.98it/s]
We will save to Disk and not RAM now.
 88%|████████▊ | 42/48 [00:02<00:00, 16.03it/s]


RuntimeError: *** Unsloth: Failed compiling llama.cpp using os.system(...) with error 2. Please report this ASAP!

In [ ]:
import torch

prompt = """You are an autonomous programmer.
ISSUE:
Fix this Python bug:

def add_numbers(a, b):
    return a - b

Your output MUST alwasy have this format:
DISCUSSION
<command>
<patch>
</command>
You MUST RETURN A PATCH.
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
import uvicorn
from pyngrok import ngrok
import nest_asyncio

# Apply asyncio patch for Colab
nest_asyncio.apply()

app = FastAPI()

# Load your model (adjust this to your model's loading code)
from unsloth import FastLanguageModel  # Or your actual import

class RequestBody(BaseModel):
    prompt: str
    max_tokens: int = 200000
    temperature: float = 0.7
    stop: str | None = None


@app.post("/v1/completions")
def generate_completion(data: RequestBody):
  inputs = tokenizer(data.prompt, return_tensors="pt").to(model.device)

  with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
        eos_token_id=tokenizer.convert_tokens_to_ids("</command>"),
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(decoded)

    return {"response": decoded}
# Open the public tunnel
ngrok.set_auth_token(userdata.get('ngrok'))
public_url = ngrok.connect(8000)
print("🔗 Public URL:", public_url)

# Start the FastAPI app
uvicorn.run(app, port=8000)


🔗 Public URL: NgrokTunnel: "https://4201-35-240-242-175.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [46371]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
